In [3]:
import pandas as pd
import numpy as np
import random
import pickle
import sys
from keras.models import load_model
from keras import backend



sys.path.append("../../src/features")
import build_features
import RFV_text_vectorizing
sys.path.append("../../src/models/train_model")
import NN_VE_model
sys.path.append("../../src/models/predict_model")
import predictor


pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_rows', None)
pd.options.display.max_columns = None

##  Read models and data

In [4]:
# read tokenization values
with open("../../models/cdc_2009_att_text_tokenizer.pickle", "rb") as f:
    tokenizer = pickle.load(f)
with open("../../models/cdc_2009_att_text_max_length.pickle", "rb") as f:
    max_seq_length= pickle.load(f) 
with open("../../models/cdc_2009_att_text_vocabulary.pickle", "rb") as f:
    vocabulary= pickle.load(f) 

In [5]:
# read model
cdc_model = load_model('../../models/cdc_2009_nn_att_text_embedding.H5',custom_objects={'backend': backend})
#cdc_model.summary()
# read model
rss_model = load_model('../../models/cdc_rss_2009_nn_att_text_embedding.H5',custom_objects={'backend': backend})
#cdc_model.summary()

/home/zenobiosky/anaconda2/lib/python2.7/site-packages/keras/engine/topology.py:1269: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [6]:
# records with vital signs = -9 had been removed from this file, for demo purposes
fileName = '../../data/processed/FOR_PREDICTIONS_2009.csv'
cdc_file = pd.read_csv(fileName)


In [7]:
#getting one record from the input file, as an example
record_index = random.randint(0,len(cdc_file))
sample_record = cdc_file.iloc[[record_index]]
sample_record.head()

,VYEAR,VMONTH,VDAYR,AGE,ARRTIME,WAITTIME,LOV,RESIDNCE,SEX,ETHUN,RACEUN,ARREMS,TEMPF,PULSE,RESPR,BPSYS,BPDIAS,POPCT,ONO2,GCS,IMMEDR,PAIN,SEEN72,RFV1,RFV2,RFV3,EPISODE,INJURY,CHF,DIABETES,LEFTBMSE,LEFTAMSE,LEFTAMA,DOA,DIEDED,TRANPSYC,TRANOTH,ADMITHOS,OBSHOS,OBSDIS,OTHDISP,ADMIT,HDSTAT,BDATEFL,IMMEDRFL,REGION,MSA
19147,2009,1,6,18,1513,137,226,1,1,-9,-9,-8,990,82,20,121,84,99,-8,-9,5,0,-8,14450,-9,-9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-7,-7,0,0,3,1


## Pass data to predictor

In [8]:
json_data = sample_record.to_json(orient='records')

In [9]:
reload(predictor)
output = predictor.predict(json_data, tokenizer, max_seq_length, vocabulary, cdc_model, rss_model)


Creating text for embeddings


In [10]:
print(output)

{"co": 0.054508499801158905, "rss": 0, "esi": 5, "co_fi": [{"input_value": "1", "featureName": "Age_18_30_range", "attention_weight": 0.025005146861076355}, {"input_value": null, "featureName": "reason2_for_visit", "attention_weight": 0.017827056348323822}, {"input_value": "1", "featureName": "Reason_Respiratory_Other", "attention_weight": 0.015303058549761772}, {"input_value": "0", "featureName": "Reason_Musculoskeletal_Other", "attention_weight": 0.014030208811163902}, {"input_value": "14450: ", "featureName": "reason1_for_visit", "attention_weight": 0.014004033990204334}, {"input_value": "0", "featureName": "Reason_Shortness_of_Breath", "attention_weight": 0.013221587985754013}, {"input_value": "0", "featureName": "chf_indicator", "attention_weight": 0.013051443733274937}, {"input_value": "-8: NOT by ambulance", "featureName": "arrival_model", "attention_weight": 0.01235964149236679}, {"input_value": "1", "featureName": "Moderate_Tachypnea", "attention_weight": 0.012284191325306892}